## Data preprocessing

This notebook performs essential preprocessing on the raw dataset to prepare it for further analysis and modeling.

### Objectives of Preprocessing:
1. **Remove missing values from all stations:** function drop_missing_values
2. **Find the common dates across all stations and remove the non-overlapping ones**: function find_common_dates

### Check the data after preprocessing:
1. **Check missing values:** function get_missing_values
2. **Check remaining dates:** function get_date
3. **Describe data after preprocessing (min, max, quantile, std):**: function describe_data
4. **Count ouliers in each feature of all stations:** function count_outliers_IQR

In [1]:
import pandas as pd

In [2]:
df_CaMau = pd.read_csv('./DATA_SENT SV/Ca Mau_Final.csv')
df_LangSon = pd.read_csv('./DATA_SENT SV/Lang Son_Final.csv')
df_LaoCai = pd.read_csv('./DATA_SENT SV/Lao Cai_Final.csv')
df_NoiBai = pd.read_csv('./DATA_SENT SV/NoiBai_Final.csv')
df_PhuBai = pd.read_csv('./DATA_SENT SV/Phu Bai_Final.csv')
df_QuyNhon = pd.read_csv('./DATA_SENT SV/Quy Nhon_Final.csv')
df_TPHCM = pd.read_csv('./DATA_SENT SV/TPHCM_Final.csv')
df_Vinh = pd.read_csv('./DATA_SENT SV/Vinh_Final.csv')

station_dfs = {
    'Noi_Bai': df_NoiBai,
    'Lang_Son': df_LangSon,
    'Lao_Cai': df_LaoCai,

    'Vinh': df_Vinh,
    'Phu_Bai': df_PhuBai,
    'Quy_Nhon': df_QuyNhon,

    'TPHCM': df_TPHCM,
    'Ca_Mau': df_CaMau
}

### Preprocessing

In [3]:
def drop_missing_values(station_dfs):
    new_station_df = dict()

    for station, df in station_dfs.items():
        new_df = df.copy()
        df_cleaned = new_df.dropna()
        new_station_df[station] = df_cleaned

    return new_station_df

none_missing_station_dfs = drop_missing_values(station_dfs)
print(none_missing_station_dfs)

{'Noi_Bai':              YMD     NAME  LATITUDE  LONGITUDE  YEAR  MONTH  DAY      TMP_2  \
0       1/1/1990  NOI BAI     21.22     105.81  1990      1    1  14.562500   
1       1/2/1990  NOI BAI     21.22     105.81  1990      1    2  16.042857   
2       1/3/1990  NOI BAI     21.22     105.81  1990      1    3  16.812500   
3       1/4/1990  NOI BAI     21.22     105.81  1990      1    4  16.300000   
4       1/5/1990  NOI BAI     21.22     105.81  1990      1    5  16.771429   
...          ...      ...       ...        ...   ...    ...  ...        ...   
12648  8/27/2024  NOI BAI     21.22     105.81  2024      8   27  30.450000   
12649  8/28/2024  NOI BAI     21.22     105.81  2024      8   28  31.708333   
12650  8/29/2024  NOI BAI     21.22     105.81  2024      8   29  31.291667   
12651  8/30/2024  NOI BAI     21.22     105.81  2024      8   30  29.250000   
12652  8/31/2024  NOI BAI     21.22     105.81  2024      8   31  30.166667   

           DEW_2         RH    AT mean 

In [4]:
def find_common_dates(station_df):
    date_sets = {}
    for station, df in station_df.items():
        date_sets[station] = set(zip(df['YEAR'], df['MONTH'], df['DAY']))

    # Tìm ngày chung giữa tất cả stations
    common_dates = set.intersection(*date_sets.values())
    print(f"\nSố ngày chung giữa tất cả stations: {len(common_dates)}")

    common_dates = sorted(list(common_dates))

    filtered_station_df = {}
    for station, df in station_df.items():
        mask = df.apply(lambda x: (x['YEAR'], x['MONTH'], x['DAY']) in common_dates, axis=1)
        filtered_station_df[station] = df[mask].copy()
        print(f"{station} sau khi lọc: {len(filtered_station_df[station])} ngày")

    return common_dates, filtered_station_df

common_dates, filtered_station_df = find_common_dates(none_missing_station_dfs)


Số ngày chung giữa tất cả stations: 11761
Noi_Bai sau khi lọc: 11761 ngày
Lang_Son sau khi lọc: 11761 ngày
Lao_Cai sau khi lọc: 11761 ngày
Vinh sau khi lọc: 11761 ngày
Phu_Bai sau khi lọc: 11761 ngày
Quy_Nhon sau khi lọc: 11761 ngày
TPHCM sau khi lọc: 11761 ngày
Ca_Mau sau khi lọc: 11761 ngày


### EDA - After preprocessing

In [5]:
def get_missing_values(station_dfs):
    for station_name, station_df in station_dfs.items():
        print(f"------STATION {station_name}------")
        print(station_df.isnull().sum())

get_missing_values(filtered_station_df)

------STATION Noi_Bai------
YMD          0
NAME         0
LATITUDE     0
LONGITUDE    0
YEAR         0
MONTH        0
DAY          0
TMP_2        0
DEW_2        0
RH           0
AT mean      0
AT max       0
dtype: int64
------STATION Lang_Son------
YMD          0
NAME         0
LATITUDE     0
LONGITUDE    0
YEAR         0
MONTH        0
DAY          0
DEW_2        0
TMP_2        0
RH           0
AT mean      0
AT max       0
dtype: int64
------STATION Lao_Cai------
YMD          0
NAME         0
LATITUDE     0
LONGITUDE    0
YEAR         0
MONTH        0
DAY          0
DEW_2        0
TMP_2        0
RH           0
AT mean      0
AT max       0
dtype: int64
------STATION Vinh------
YMD          0
NAME         0
LATITUDE     0
LONGITUDE    0
YEAR         0
MONTH        0
DAY          0
TMP_2        0
DEW_2        0
RH           0
AT mean      0
AT max       0
dtype: int64
------STATION Phu_Bai------
YMD          0
NAME         0
LATITUDE     0
LONGITUDE    0
YEAR         0
MONTH        0


In [6]:
def get_date(station_dfs):
    for station_name, station_df in station_dfs.items():
        ymd = pd.to_datetime(station_df[['YEAR', 'MONTH', 'DAY']])

        print(f"------STATION {station_name.upper()}------")
        print(f"First date: {ymd.min()}")
        print(f"Last date: {ymd.max()}")
        print(f"Years: {ymd.dt.year.max() - ymd.dt.year.min()}")
        print(f"Days: {len(station_df)}")

get_date(filtered_station_df)

------STATION NOI_BAI------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION LANG_SON------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION LAO_CAI------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION VINH------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION PHU_BAI------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION QUY_NHON------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION TPHCM------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761
------STATION CA_MAU------
First date: 1992-04-01 00:00:00
Last date: 2024-08-31 00:00:00
Years: 32
Days: 11761


In [7]:
def describe_data(station_dfs):
    for station_name, station_df in station_dfs.items():
        print(f"------STATION {station_name.upper()}------")
        print(station_df.describe())

describe_data(filtered_station_df)

------STATION NOI_BAI------
           LATITUDE     LONGITUDE          YEAR         MONTH           DAY  \
count  1.176100e+04  1.176100e+04  11761.000000  11761.000000  11761.000000   
mean   2.122000e+01  1.058100e+02   2007.976618      6.516538     15.729955   
std    3.552865e-15  2.842292e-14      9.343850      3.432352      8.797574   
min    2.122000e+01  1.058100e+02   1992.000000      1.000000      1.000000   
25%    2.122000e+01  1.058100e+02   2000.000000      4.000000      8.000000   
50%    2.122000e+01  1.058100e+02   2008.000000      7.000000     16.000000   
75%    2.122000e+01  1.058100e+02   2016.000000      9.000000     23.000000   
max    2.122000e+01  1.058100e+02   2024.000000     12.000000     31.000000   

              TMP_2         DEW_2            RH       AT mean        AT max  
count  11761.000000  11761.000000  11761.000000  11761.000000  11761.000000  
mean      24.416541     20.101076     78.425807     21.924588     25.327688  
std        5.211136      5

In [8]:
def count_outliers_IQR(station_dfs):
    for station_name, station_df in station_dfs.items():
        print(f"------STATION {station_name.upper()}------")
        outlier_counts = dict()

        for col in station_df.select_dtypes(include=['float64', 'int64']):
            Q1 = station_df[col].quantile(0.25)
            Q3 = station_df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = station_df[(station_df[col] < lower_bound) | (station_df[col] > upper_bound)]
            outlier_counts[col] = len(outliers)
            
        print(outlier_counts)

count_outliers_IQR(filtered_station_df)

------STATION NOI_BAI------
{'LATITUDE': 0, 'LONGITUDE': 0, 'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'TMP_2': 4, 'DEW_2': 98, 'RH': 282, 'AT mean': 4, 'AT max': 12}
------STATION LANG_SON------
{'LATITUDE': 0, 'LONGITUDE': 0, 'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'DEW_2': 0, 'TMP_2': 6, 'RH': 217, 'AT mean': 6, 'AT max': 37}
------STATION LAO_CAI------
{'LATITUDE': 0, 'LONGITUDE': 0, 'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'DEW_2': 15, 'TMP_2': 4, 'RH': 128, 'AT mean': 4, 'AT max': 8}
------STATION VINH------
{'LATITUDE': 0, 'LONGITUDE': 0, 'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'TMP_2': 4, 'DEW_2': 187, 'RH': 243, 'AT mean': 4, 'AT max': 4}
------STATION PHU_BAI------
{'LATITUDE': 0, 'LONGITUDE': 0, 'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'TMP_2': 13, 'DEW_2': 346, 'RH': 111, 'AT mean': 13, 'AT max': 11}
------STATION QUY_NHON------
{'LATITUDE': 0, 'LONGITUDE': 0, 'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'TMP_2': 6, 'DEW_2': 301, 'RH': 497, 'AT mean': 6, 'AT max': 3}
------STATION TPHCM------
{'LATITUDE': 0, 'LONGITUDE': 0, 